In [ ]:
pip install dask

In [ ]:
import dask.dataframe as dd
from google.colab import files


In [ ]:
customers_dd = dd.read_csv("blinkit_customers.csv")
orders_dd = dd.read_csv("blinkit_orders.csv")
order_items_dd = dd.read_csv("blinkit_order_items.csv")
products_dd = dd.read_csv("blinkit_products.csv")
inventory_dd = dd.read_csv("blinkit_inventory.csv")
delivery_dd = dd.read_csv("blinkit_delivery_performance.csv")
feedback_dd = dd.read_csv("blinkit_customer_feedback.csv")


In [ ]:
orders_dd['order_date'] = dd.to_datetime(orders_dd['order_date'], errors='coerce')
orders_dd['promised_delivery_time'] = dd.to_datetime(orders_dd['promised_delivery_time'], errors='coerce')
orders_dd['actual_delivery_time'] = dd.to_datetime(orders_dd['actual_delivery_time'], errors='coerce')

customers_dd['registration_date'] = dd.to_datetime(customers_dd['registration_date'], errors='coerce')
feedback_dd['feedback_date'] = dd.to_datetime(feedback_dd['feedback_date'], errors='coerce')
inventory_dd['date'] = dd.to_datetime(inventory_dd['date'], format='%d-%m-%Y', errors='coerce')


In [ ]:
# Merge orders with customers
merged_orders_dd = orders_dd.merge(customers_dd, on='customer_id', how='left')

# Merge order_items with products and orders
merged_items_dd = order_items_dd.merge(products_dd, on='product_id', how='left') \
                                .merge(orders_dd, on='order_id', how='left')

# Merge feedback with customer data
merged_feedback_dd = feedback_dd.merge(customers_dd, on='customer_id', how='left')


/usr/local/lib/python3.11/dist-packages/dask/dataframe/multi.py:169: UserWarning: Merging dataframes with merge column data type mismatches: 
+--------------------------+------------+-------------+
| Merge columns            | left dtype | right dtype |
+--------------------------+------------+-------------+
| ('order_id', 'order_id') | int64      | float64     |
+--------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


In [ ]:
#  1. Monthly Order Trend (Order Volume over Time)

order_trend_dd = (
    merged_orders_dd
    .assign(month=merged_orders_dd['order_date'].dt.to_period('M').astype(str))
    .groupby('month')['order_id']
    .count()
    .reset_index()
    .rename(columns={'order_id': 'order_count'})
)

order_trend_dd.compute().head()

,month,order_count
0,2023-01,1399
1,2023-02,1341
2,2023-03,1453
3,2023-04,1381
4,2023-05,1435


In [ ]:
# 2. Top 10 Most Ordered Products

top_products_dd = (
    merged_items_dd
    .groupby(['product_id', 'product_name', 'category'])['quantity']
    .sum()
    .reset_index()
    .nlargest(10, 'quantity')
)

top_products_dd.compute()

top_products_dd.compute()

,product_id,product_name,category,quantity
195,842190,ipsum impedit,Cold Drinks & Juices,2132
216,913328,quasi corrupti,Fruits & Vegetables,2095
81,319394,quibusdam in,Baby Care,2073
92,357493,provident sapiente,Baby Care,2072
96,385835,voluptas earum,Grocery & Staples,2068
67,277760,maiores eveniet,Cold Drinks & Juices,2048
124,522793,velit eius,Pet Care,2037
167,697653,qui vero,Snacks & Munchies,2035
204,872566,corporis repudiandae,Baby Care,2034
62,266697,dicta vero,Grocery & Staples,2033


In [ ]:
# 3. Customer Segment Performance

customer_segment_summary_dd = (
    customers_dd
    .groupby('customer_segment')[['total_orders', 'avg_order_value']]
    .mean()
    .reset_index()
)

customer_segment_summary_dd.compute()


,customer_segment,total_orders,avg_order_value
0,Inactive,10.514109,1106.047452
1,New,10.514144,1100.784109
2,Premium,10.560500,1097.730387
3,Regular,10.576617,1112.019780


In [ ]:
#  4. Feedback Sentiment Distribution

sentiment_distribution_dd = (
    feedback_dd['sentiment']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'sentiment', 'sentiment': 'count'})
)

sentiment_distribution_dd.compute()

,count,count
0,Negative,4991
1,Neutral,9893
2,Positive,35116


In [ ]:
#  5. Delivery Status Breakdown

delivery_status_summary_dd = (
    orders_dd['delivery_status']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'delivery_status', 'delivery_status': 'count'})
)

delivery_status_summary_dd.compute()

,count,count
0,On Time,35116
1,Significantly Delayed,4991
2,Slightly Delayed,9893


In [ ]:
# 6. Top 10 Products with Highest Inventory Damage %

inventory_dd['damage_percent'] = (
    (inventory_dd['damaged_stock'] / inventory_dd['stock_received']).fillna(0) * 100
)

avg_damage_by_product_dd = (
    inventory_dd
    .groupby('product_id')['damage_percent']
    .mean()
    .reset_index()
    .nlargest(10, 'damage_percent')
)

avg_damage_by_product_dd.compute()

,product_id,damage_percent
0,4104,inf
1,10203,inf
2,25710,inf
3,28011,inf
4,29286,inf
5,38911,inf
6,39812,inf
7,46310,inf
8,49731,inf
9,59398,inf


In [ ]:
# 7. Top 10 areas by Orders

top_areas_dd = (
    merged_orders_dd
    .groupby('area')['order_id']
    .count()
    .reset_index()
    .rename(columns={'order_id': 'total_orders'})
    .nlargest(10, 'total_orders')
)

top_areas_dd.compute()

,area,total_orders
100,Ghaziabad,309
24,Aurangabad,306
155,Kharagpur,221
5,Aizawl,214
99,Gaya,210
202,Nagaon,207
52,Bhopal,206
105,Gulbarga,206
123,Ichalkaranji,204
278,Sri Ganganagar,203


In [ ]:
# 8. New vs. Returning Customers

# Define new customers as those with only 1 order
customer_order_counts_dd = (
    merged_orders_dd
    .groupby('customer_id')['order_id']
    .count()
    .reset_index()
    .rename(columns={'order_id': 'order_count'})
)

# Label customer type
customer_order_counts_dd['customer_type'] = customer_order_counts_dd['order_count'].apply(
    lambda x: 'New' if x == 1 else 'Returning', meta=('order_count', 'object')
)

# Count new vs returning
new_vs_returning_dd = (
    customer_order_counts_dd
    .groupby('customer_type')['customer_id']
    .count()
    .reset_index()
    .rename(columns={'customer_id': 'count'})
)

new_vs_returning_dd.compute()


,customer_type,count
0,New,6852
1,Returning,14819


In [ ]:
# 9. Top Segments by Avg Order Value

top_segments_dd = (
    customers_dd
    .groupby('customer_segment')['avg_order_value']
    .mean()
    .reset_index()
    .nlargest(10, 'avg_order_value')
)

top_segments_dd.compute()

,customer_segment,avg_order_value
3,Regular,1112.019780
0,Inactive,1106.047452
1,New,1100.784109
2,Premium,1097.730387
